In [1]:
import pandas as pd, numpy, math
documentsFilename = 'estadao_noticias_eleicao.csv'
gabaritoFilename = 'gabarito.csv'
data = pd.read_csv(documentsFilename)
gabarito = pd.read_csv(gabaritoFilename)

In [2]:
#Function to print topFive of documents
def returnTopFive(topFive, search, query):
    arrayString = []
    print("Top five documents of " + search + "  for query '" + query + "'")
    intArray = []
    for document in sorted(topFive, key = topFive.get, reverse=True):
        print ("Document: " + str(document) + ", frequence: " + str(topFive[document]))
        intArray.append(document)

    #Convert array of int to array of strings
    for number in intArray:
        arrayString.append(str(number))
        
    print(arrayString)
    return arrayString

#Function to verify if element 's' is nan    
def is_nan(s):
    try:
        float(s)
        return True
    except ValueError:
        return False

In [3]:
#Function to search with binary method
def searchBinary(query):
    searchWords = query.split(" ")
    #The top five document by higher frequences 
    topFive = {}
    for i in range(len(data.values)):
        #Get frequence of document i
        documentID, frequence = frequenceBinary(searchWords, data.values[i]);
        
        #Add document and frequence in topFive dictionary
        if(len(topFive) < 5):
            topFive[documentID]= frequence
        else:
            minKey = None
            for j in topFive.keys():
                if(frequence > topFive[j]):
                    if(minKey == None or topFive[j] < topFive[minKey]):
                        minKey = j
            if(minKey != None):
                topFive[documentID] = frequence;
                del topFive[minKey]
    
    return returnTopFive(topFive, "binary search", query)
        
#Function to calculate the frequency of query words in the document for binary search
def frequenceBinary(query, document):
    frequenceBinary = {}
    if(is_nan(document[1])):
        document[1] = ""
    if(is_nan(document[2])):
        document[2] = ""
    if(is_nan(document[3])):
        document[3] = ""
    documentWords = document[1].split(" ") + document[2].split(" ") + document[3].split(" ")
    
    for q in query:
        #Iterate on all document words
        for word in documentWords:
            if(q.lower() == word.lower()):
                if(q not in frequenceBinary):
                    frequenceBinary[q] = 1
                    break
    frequence = 0
    for freq in frequenceBinary:
        frequence += frequenceBinary[freq]
    
    return document[5], frequence
   

In [4]:
#Function to search with TF method 
def searchTF(query):
    searchWords = query.split(" ")
    #The top five document by higher frequences 
    topFive = {}
    for i in range(len(data.values)):
        #Get frequence of document i
        documentID, frequence = frequenceTF(searchWords, data.values[i]);
        
        #Add document and frequence in topFive dictionary
        if(len(topFive) < 5):
            topFive[documentID]= frequence
        else:
            minKey = None
            for j in topFive.keys():
                if(frequence > topFive[j]):
                    if(minKey == None or topFive[j] < topFive[minKey]):
                        minKey = j
            if(minKey != None):
                topFive[documentID] = frequence;
                del topFive[minKey]
    
    return returnTopFive(topFive, "TF search", query)
        
#Function to calculate the frequency of query words in the document for TF search
def frequenceTF(query, document):
    frequence = 0
    if(is_nan(document[1])):
        document[1] = ""
    if(is_nan(document[2])):
        document[2] = ""
    if(is_nan(document[3])):
        document[3] = ""
    documentWords = document[1].split(" ") + document[2].split(" ") + document[3].split(" ")
    
    #Iterate on all document words
    for word in documentWords:
        for q in query:
            if(q.lower() == word.lower()):
                frequence += 1
    
    return document[5], frequence
   

In [5]:
#Function to search with TF-IDF method 
def searchTF_IDF(query):
    searchWords = query.split(" ")
    
    #Number of documents in file "data"
    numberOfDocs = len(data.values)
    
    #The top five document by higher frequences 
    topFive = {}
    
    #Calculate the number of documents that query words appear
    dictDocsFrequence = {}
    for i in range(len(data.values)):
        dictDocsFrequence = IDF(searchWords, data.values[i], dictDocsFrequence)
    
    #Calculate IDF of query words
    dictIDF = {}
    for i in dictDocsFrequence.keys(): 
        dictIDF[i] = math.log10((numberOfDocs + 1) / dictDocsFrequence[i])
    
    #Iterate in all documents to calculate frequence of query words
    for i in range(len(data.values)):
        #Get frequence of document i
        documentID, frequence = frequenceTF_IDF(searchWords, data.values[i], dictIDF);
        
        #Add document and frequence in topFive dictionary
        if(len(topFive) < 5):
            topFive[documentID]= frequence
        else:
            minKey = None
            for j in topFive.keys():
                if(frequence > topFive[j]):
                    if(minKey == None or topFive[j] < topFive[minKey]):
                        minKey = j
            if(minKey != None):
                topFive[documentID] = frequence;
                del topFive[minKey]
    
    return returnTopFive(topFive, "TF-IDF search", query)

#Function to calculate IDF of query words
def IDF(query, document, dictDocsFrequence):
    #Verify if the line is nan
    if(is_nan(document[1])):
        document[1] = " "
    if(is_nan(document[2])):
        document[2] = " "
    if(is_nan(document[3])):
        document[3] = " "
    
    #All the words of documents in array
    documentWords = document[1].split(" ") + document[2].split(" ") + document[3].split(" ")
    
    #Iterate on query words
    for wordQuery in query:
        #Iterate on document words
        for wordDocuments in documentWords:
            #Verify if exist the word in document and change the frequence of documents
            if(wordQuery.lower() == wordDocuments.lower()):
                if(wordQuery in dictDocsFrequence):
                    dictDocsFrequence[wordQuery] = dictDocsFrequence[wordQuery] + 1
                    break
                else:
                    dictDocsFrequence[wordQuery] = 1
                    break

    return dictDocsFrequence

#Function to calculate the frequency of query words in the document multiplied by IDF
def frequenceTF_IDF(query, document, dictIDF):
    frequence = 0
    
    #Verify if the line is nan
    if(is_nan(document[1])):
        document[1] = ""
    if(is_nan(document[2])):
        document[2] = ""
    if(is_nan(document[3])):
        document[3] = ""
        
    #All the words of documents in array
    documentWords = document[1].split(" ") + document[2].split(" ") + document[3].split(" ")
    
    #Iterate on query words
    for wordQuery in query:
        frequenceWord = 0
        #Iterate on document words
        for wordDocument in documentWords:
            if(wordQuery.lower() == wordDocument.lower()):
                frequenceWord += 1
        #Calculate frequence with IDF
        frequence += frequenceWord * dictIDF[wordQuery]
        
    return document[5], frequence

In [6]:
def search_BM25(query):
    searchWords = query.split(" ")
    
    #Number of documents in file "data"
    numberOfDocs = len(data.values)
    
    #The top five document by higher frequences 
    topFive = {}
    
    #Calculate the number of documents that query words appear
    dictDocsFrequence = {}
    for i in range(len(data.values)):
        dictDocsFrequence = IDF(searchWords, data.values[i], dictDocsFrequence)
        
    #Calculate IDF of query words
    dictIDF = {}
    for i in dictDocsFrequence.keys(): 
        dictIDF[i] = math.log10((numberOfDocs + 1) / dictDocsFrequence[i])
    
    #Iterate in all documents to calculate frequence of query words
    for i in range(len(data.values)):
        #Get frequence of document i
        documentID, frequence = frequenceBM25(searchWords, data.values[i], dictIDF);
        
        #Add document and frequence in topFive dictionary
        if(len(topFive) < 5):
            topFive[documentID]= frequence
        else:
            minKey = None
            for j in topFive.keys():
                if(frequence > topFive[j]):
                    if(minKey == None or topFive[j] < topFive[minKey]):
                        minKey = j
            if(minKey != None):
                topFive[documentID] = frequence;
                del topFive[minKey]
    
    return returnTopFive(topFive, "BM25 search", query)
    
#Function to calculate IDF of query words
def IDF(query, document, dictDocsFrequence):
    #Verify if the line is nan
    if(is_nan(document[1])):
        document[1] = " "
    if(is_nan(document[2])):
        document[2] = " "
    if(is_nan(document[3])):
        document[3] = " "
    
    #All the words of documents in array
    documentWords = document[1].split(" ") + document[2].split(" ") + document[3].split(" ")
    
    #Iterate on query words
    for wordQuery in query:
        #Iterate on document words
        for wordDocuments in documentWords:
            #Verify if exist the word in document and change the frequence of documents
            if(wordQuery.lower() == wordDocuments.lower()):
                if(wordQuery in dictDocsFrequence):
                    dictDocsFrequence[wordQuery] = dictDocsFrequence[wordQuery] + 1
                    break
                else:
                    dictDocsFrequence[wordQuery] = 1
                    break

    return dictDocsFrequence

#Function to calculate the frequency of query words in the document  
def frequenceBM25(query, document, dictIDF):
    frequence = 0
    k = 5
    
    #Verify if the line is nan
    if(is_nan(document[1])):
        document[1] = ""
    if(is_nan(document[2])):
        document[2] = ""
    if(is_nan(document[3])):
        document[3] = ""
        
    #All the words of documents in array
    documentWords = document[1].split(" ") + document[2].split(" ") + document[3].split(" ")
    
    #Iterate on query words
    for wordQuery in query:
        frequenceWord = 0
        #Iterate on document words
        for wordDocument in documentWords:
            if(wordQuery.lower() == wordDocument.lower()):
                frequenceWord += 1
        #Calculate frequence with IDF
        frequence += (((k+1) * frequenceWord)/(frequenceWord + k)) * dictIDF[wordQuery]
        
    return document[5], frequence
    

In [7]:
def apk(actual, predicted, k=10):
    """
    Computes the average precision at k.
    This function computes the average prescision at k between two lists of
    items.
    Parameters
    ----------
    actual : list
             A list of elements that are to be predicted (order doesn't matter)
    predicted : list
                A list of predicted elements (order does matter)
    k : int, optional
        The maximum number of predicted elements
    Returns
    -------
    score : double
            The average precision at k over the input lists
    """
    if len(predicted)>k:
        predicted = predicted[:k]

    score = 0.0
    num_hits = 0.0

    for i,p in enumerate(predicted):
        if p in actual and p not in predicted[:i]:
            num_hits += 1.0
            score += num_hits / (i+1.0)

    if not actual:
        return 0.0

    return score / min(len(actual), k)

def mapk(actual, predicted, k=10):
    """
    Computes the mean average precision at k.
    This function computes the mean average prescision at k between two lists
    of lists of items.
    Parameters
    ----------
    actual : list
             A list of lists of elements that are to be predicted 
             (order doesn't matter in the lists)
    predicted : list
                A list of lists of predicted elements
                (order matters in the lists)
    k : int, optional
        The maximum number of predicted elements
    Returns
    -------
    score : double
            The mean average precision at k over the input lists
    """
    return numpy.mean([apk(a,p,k) for a,p in zip(actual, predicted)])

In [8]:
#Search to query 'segundo turno'
busca_binaria = searchBinary("segundo turno")
print(mapk(gabarito.values[0][2], busca_binaria, k=5))
print(mapk(gabarito.values[0][1], busca_binaria, k=5))
print ("\n")

busca_TF = searchTF("segundo turno")
print(mapk(gabarito.values[1][2], busca_TF, k=5))
print(mapk(gabarito.values[1][1], busca_TF, k=5))
print ("\n")

busca_TF_IDF = searchTF_IDF("segundo turno")
print(mapk(gabarito.values[2][2], busca_TF_IDF, k=5))
print(mapk(gabarito.values[2][1], busca_TF_IDF, k=5))
print ("\n")

busca_BM25 = search_BM25("segundo turno")
print(mapk(gabarito.values[3][2], busca_BM25, k=5))
print(mapk(gabarito.values[3][1], busca_BM25, k=5))
print ("\n")

#_________________________________________________________

#Search to query 'lava jato'

busca_binaria = searchBinary("lava jato")
print(mapk(gabarito.values[0][2], busca_binaria, k=5))
print(mapk(gabarito.values[0][1], busca_binaria, k=5))
print ("\n")

busca_TF = searchTF("lava jato")
print(mapk(gabarito.values[1][2], busca_TF, k=5))
print(mapk(gabarito.values[1][1], busca_TF, k=5))
print ("\n")

busca_TF_IDF = searchTF_IDF("lava jato")
print(mapk(gabarito.values[2][2], busca_TF_IDF, k=5))
print(mapk(gabarito.values[2][1], busca_TF_IDF, k=5))
print ("\n")

busca_BM25 = search_BM25("lava jato")
print(mapk(gabarito.values[3][2], busca_BM25, k=5))
print(mapk(gabarito.values[3][1], busca_BM25, k=5))
print ("\n")

#_________________________________________________________

#Search to query 'projeto de lei'

busca_binaria = searchBinary("projeto de lei")
print(mapk(gabarito.values[0][2], busca_binaria, k=5))
print(mapk(gabarito.values[0][1], busca_binaria, k=5))
print ("\n")

busca_TF = searchTF("projeto de lei")
print(mapk(gabarito.values[1][2], busca_TF, k=5))
print(mapk(gabarito.values[1][1], busca_TF, k=5))
print ("\n")

busca_TF_IDF = searchTF_IDF("projeto de lei")
print(mapk(gabarito.values[2][2], busca_TF_IDF, k=5))
print(mapk(gabarito.values[2][1], busca_TF_IDF, k=5))
print ("\n")

busca_BM25 = search_BM25("projeto de lei")
print(mapk(gabarito.values[3][2], busca_BM25, k=5))
print(mapk(gabarito.values[3][1], busca_BM25, k=5))
print ("\n")

#_________________________________________________________

#Search to query 'compra de voto'

busca_binaria = searchBinary("compra de voto")
print(mapk(gabarito.values[0][2], busca_binaria, k=5))
print(mapk(gabarito.values[0][1], busca_binaria, k=5))
print ("\n")

busca_TF = searchTF("compra de voto")
print(mapk(gabarito.values[1][2], busca_TF, k=5))
print(mapk(gabarito.values[1][1], busca_TF, k=5))
print ("\n")

busca_TF_IDF = searchTF_IDF("compra de voto")
print(mapk(gabarito.values[2][2], busca_TF_IDF, k=5))
print(mapk(gabarito.values[2][1], busca_TF_IDF, k=5))
print ("\n")

busca_BM25 = search_BM25("compra de voto")
print(mapk(gabarito.values[3][2], busca_BM25, k=5))
print(mapk(gabarito.values[3][1], busca_BM25, k=5))
print ("\n")

#_________________________________________________________

#Search to query 'ministério público'

busca_binaria = searchBinary("ministério público")
print(mapk(gabarito.values[0][2], busca_binaria, k=5))
print(mapk(gabarito.values[0][1], busca_binaria, k=5))
print ("\n")

busca_TF = searchTF("ministério público")
print(mapk(gabarito.values[1][2], busca_TF, k=5))
print(mapk(gabarito.values[1][1], busca_TF, k=5))
print ("\n")

busca_TF_IDF = searchTF_IDF("ministério público")
print(mapk(gabarito.values[2][2], busca_TF_IDF, k=5))
print(mapk(gabarito.values[2][1], busca_TF_IDF, k=5))
print ("\n")

busca_BM25 = search_BM25("ministério público")
print(mapk(gabarito.values[3][2], busca_BM25, k=5))
print(mapk(gabarito.values[3][1], busca_BM25, k=5))
print ("\n")

#_________________________________________________________

#Avaliação final
print("De todas as buscas vemos que o método binário é muito abstrato, onde a busca que faz no documento é apenas pra informar se existe as palavras procuradas ou não (zero ou um), o retorno dos 5 documentos será sempre os que aparecem todas as palavras da consulta pelo menos uma vez, ou seja a frequência máxima será sempre o número de palavras da consulta")

print("A busca TF é mais eficaz que a busca binária, porém não faz uma análise para quando a palavra é repetida várias vezes no mesmo documento, deixando assim documentos com palavras pouco importantes (como 'de', 'com', etc) no topo da busca")

print("Para o TF-IDF ele já não possui o mesmo erro que o TF mostra, fazendo contagem da frequencia da palavra em todos os documentos para que tenhamos uma noção de quão importante é essa palavra para a consulta (ela se mostra uma palavra mais comum)")


Top five documents of binary search  for query 'segundo turno'
Document: 7, frequence: 2
Document: 69, frequence: 2
Document: 92, frequence: 2
Document: 102, frequence: 2
Document: 111, frequence: 2
['7', '69', '92', '102', '111']
0.0
0.0


Top five documents of TF search  for query 'segundo turno'
Document: 2744, frequence: 31
Document: 7, frequence: 27
Document: 2112, frequence: 17
Document: 2388, frequence: 17
Document: 4931, frequence: 15
['2744', '7', '2112', '2388', '4931']
0.05
0.1


Top five documents of TF-IDF search  for query 'segundo turno'
Document: 2744, frequence: 19.599411535630455
Document: 7, frequence: 11.950610334779924
Document: 1917, frequence: 8.99470280158626
Document: 2112, frequence: 8.700126441038062
Document: 2388, frequence: 8.065265676702682
['2744', '7', '1917', '2112', '2388']
0.0
0.1


Top five documents of BM25 search  for query 'segundo turno'
Document: 2744, frequence: 5.805526685975037
Document: 1917, frequence: 4.49735140079313
Document: 7, frequen